In [12]:
!pip install fastapi uvicorn pyngrok diffusers transformers accelerate safetensors torch

In [13]:
import torch
from diffusers import StableDiffusionPipeline
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.responses import FileResponse
from PIL import Image
import uvicorn
import os
import nest_asyncio

In [14]:
# Load Stable Diffusion v1.5
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
)
pipe.to("cuda")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.32.2",
  "_name_or_path": "runwayml/stable-diffusion-v1-5",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [17]:
# Define FastAPI app
app = FastAPI()

# Request model
class PromptRequest(BaseModel):
    prompt: str

# Endpoint to generate image
@app.post("/generate")
def generate_image(request: PromptRequest):
    image = pipe(prompt=request.prompt).images[0]
    image_path = "generated_image.png"
    image.save(image_path)
    return FileResponse(image_path, media_type="image/png")

# Fix event loop issue in Colab
nest_asyncio.apply()

# Run the API in Colab
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [2914]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     34.169.114.242:0 - "POST /generate HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2914]


In [16]:
!pip install pyngrok
from pyngrok import ngrok

# Set your Ngrok auth token (Run only once)
!ngrok config add-authtoken NGROK_AUTH_TOKEN

# Start Ngrok
ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: https://66db-34-142-160-57.ngrok-free.app
